**목표 : 타이타닉 호에서 탑승했던 사람들의 정보를 바탕으로 생존자를 예측하기**

데이터 불러오기

In [142]:
import numpy as np 
import pandas as pd 

train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data.head()  

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


각 속성의 의미

PassengerId : 각 승객의 고유 식별자  
Survived : 목표 변수로, 0은 사망, 1은 생존을 의미  
Pclass : 승객 등급  -
Name: 이름  
Sex: 성별  -
Age: 나이  *
SibSp : 타이타닉호에 탑승한 형제자매와 배우자의 수  *
Parch : 타이타닉호에 탑승한 자녀와 부모의 수  *
Ticket : 티켓 ID  
Fare : 티켓 요금(파운드 기준)  *
Cabin : 승객의 객실 번호  
Embarked : 승객이 탑승한 항구(탑승지)-

In [ ]:
#PassengerId열을 인덱스열로 만들어 데이터 누락 확인
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")
train_data.info()

Age, Cabin, Embarked에서 데이터의 누락이 있는 것을 확인하였고  
특히,  Cabin가 77%가 결측치이기에 Cabin을 제외하고,
생존율과 관계없이 승객의 탑승정보만을 갖고 있는 Name과 Ticket도 제외하겠다.

그러면, Age와 Embarked의 결측치를 처리해야하는데 데이터들의 통계를 먼저 본 뒤에    
Embarked는 장소를 가르키기에 사람이 제일 많이 타는 장소로 결측치를 대체하고  
Age는 전체승객의 평균 나이로 대체한다.

In [ ]:
#수치형 데이터로 본 통계(속성)
train_data.describe()

In [ ]:
#범주형 데이터로 본 통계들
train_data["Survived"].value_counts()

In [ ]:
train_data["Pclass"].value_counts()

In [ ]:
train_data["Sex"].value_counts()

In [ ]:
train_data["Embarked"].value_counts()

탑승장소  
S : 사우샘프턴, C : 셰르부르, Q : 퀸스타운

In [ ]:
#탑승자가 제일 많은 S로 Embarked 결측치 대체
train_data['Embarked'].fillna('S')

# 결과 확인
print(train_data['Embarked'].isnull().sum())  # 결측치가 0이어야 함

In [ ]:
#전체 탑승자의 평균 나이로 Age 결측치 대체
average_age = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(average_age)

# 결과 확인
print(train_data['Age'].isnull().sum())  # 결측치가 0이어야 함

In [ ]:
#전처리 파이프라인 만들전처리 파이프라인 만들기

#수치형 파이프라인
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

#범주형 파이프라인
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

cat_pipeline = Pipeline([
        ("ordinal_encoder", OrdinalEncoder()),
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False)),
    ])

#수치형과 범주형 파이프라인 결합
from sklearn.compose import ColumnTransformer

num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
#라벨처리
X_train = preprocess_pipeline.fit_transform(train_data)
y_train = train_data["Survived"]

생존율과 훈련데이터로 분류기를 만들어 훈련

In [ ]:
from sklearn.ensemble import RandomForestClassifier


forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

In [ ]:
#훈련된 모델을 통해 테스트 세트에 대한 예측
X_test = preprocess_pipeline.transform(test_data)
y_pred = forest_clf.predict(X_test)

In [ ]:
#교차 검증을 통해 모델의 평균 성능 계산
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

In [ ]:
submission = pd.DataFrame({
    "PassengerId": test_data.index,
    "Survived": y_pred
})

submission.to_csv('submission.csv', index=False)

In [ ]:
import os
print(os.listdir('/kaggle/working/'))